# 0. Install and Import dependencies

We will use the most advanced environment for RL-trading task - DI-engine/gym-anytrade. It's a part of DI-engine RL framework, which contains a lot of environments inside.

Important note - tensorflow==1.15.0 works only with Python 3.7 and below

In [ ]:
# !git clone https://github.com/opendilab/DI-engine.git
%pip install DI-engine/.

%pip install stable-baselines finta tensorflow-gpu==1.15.0 tensorflow==1.15.0 

Installation Check

In [ ]:
import gym

# Stable baselines - rl stuff.
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

# Processing libraries.
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Finta is needed to calculate Market Custom indicators.
from finta import TA

import ding
print(ding.__version__)

# 1. Data preprocessing

In [ ]:
# Download some data, if you need
!wget https://github.com/AminHP/gym-anytrading/blob/master/gym_anytrading/datasets/data/STOCKS_GOOGL.csv

In [ ]:
import os

csv_name = 'STOCKS_GOOGL.csv'

# check the ding installation folder
ding_folder = ding.__file__.split('ding')[0]
data_folder = os.path.join(ding_folder, 'dizoo/gym_anytrading/envs/data/STOCKS_GOOGL.csv')  

# move data to correct folder
os.replace(f"{csv_name}", f"{data_folder}")

In [ ]:
# Enhance with custom indicators using FINTA

# 2. Create new environment


In [ ]:
import numpy as np
from easydict import EasyDict
from dizoo.gym_anytrading.envs import StocksEnv

env = StocksEnv(EasyDict({"env_id": 'stocks-v0', "eps_length": 300,\
            "window_size": 20, "train_range": None, "test_range": None, "stocks_data_filename": 'STOCKS_GOOGL'}))

In [ ]:
env

### Indicators can be added by overriding _process_data method.

# 3. Build Environment and Train

In [ ]:
env = DummyVecEnv([env])
model = A2C('MlpLstmPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)

# Next tasks:
1) Fix package bugs